# 6.12 テキスト分析(3) word2vec モデル
word2vecでは、ニューラルネットワークを用い、ある語と周辺語の対応関係を示す重み付け行列を作成することで、ある特定の語群の中の語の特性をベクトル化することが可能になります。語句を数値で示すことが可能になるため、例えば語と語の類似度について、数値にて示すことが可能になります。

まず、前節の前節のコラムにて利用したDBpedia.org由来の経営学者リストについて、MeCabを用いた分かち書きを行います。

In [1]:
!pip install SPARQLWrapper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.4/565.4 kB 9.9 MB/s eta 0:00:00


In [2]:
# DBpedia.orgから日本の経営学者の情報を取得する

from SPARQLWrapper import SPARQLWrapper
import codecs

#
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

#日本の経済学者の情報をまとめて取得する
sparql2 = SPARQLWrapper(endpoint='http://ja.dbpedia.org/sparql', returnFormat='json')
sparql2.setQuery("""
PREFIX dbpedia-owl: <http://dbpedia.org/ontology/>

select distinct ?name ?abstract where {
    ?company <http://dbpedia.org/ontology/wikiPageWikiLink> <http://ja.dbpedia.org/resource/Category:日本の経営学者> .
    ?company rdfs:label ?name .
    ?company <http://dbpedia.org/ontology/abstract> ?abstract .
}
""")
results = []
results2 = sparql2.query().convert()

# このままresult2をprilltするとjson形式のデータで出力されるため、nameおよび bstractのvalueのみ取り出すことにしましょう。
# 結果を整形
for result in results2["results"]["bindings"]:
#    print("経済",";",result["name"]["value"],";",result["abstract"]["value"])
    results.append(result["abstract"]["value"])
# これにより,Wikipedia(DBPedia.org)に掲載された経済学者の名前およびその概要に関する情報を一括して取得することが出来ました
#このリストをtextに変換。リスト要素の区切りは改行とする。
text = '\n'.join(results)

f = codecs.open('経営学者リスト_dbpedia.txt', 'w', 'utf8')
f.write(text)
f.close()

In [3]:
!pip install mecab-python3
!pip install unidic-lite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.2/591.2 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 18.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658817 sha256=46816ff7fa2e7f28758c94d8822e44148c746e6b4cee639e20b0cc9a42d4fd45
  Stored in directory: /root/.cache/pip/wheels/5e/1f/0f/4d43887e5476d956fae828ee9b6687becd5544d68b51ed633d
Successfully built unidic-lite


In [4]:
# 取得した経営学者の情報を分かち書きにする

import MeCab
import codecs

tagger = MeCab.Tagger('-O wakati')

fi = codecs.open('経営学者リスト_dbpedia.txt', 'r', 'utf8')
fo = codecs.open('経営学者リスト_dbpedia_wakachi.txt', 'w', 'utf8')

line = fi.readline()

while line:
    result = tagger.parse("　"+line)
    fo.write(result[1:])
    line = fi.readline()

fi.close()
fo.close()

In [6]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 60.2 MB/s eta 0:00:00


In [7]:
#word2vec モデルを構築する
#分かち書きしたファイルを読み込み，学習モデルを作成する

# gensimのインストールが必要
# anaconda prompt で以下のコマンドを実行
# conda install gensim
# 途中で　Proceed ([y]/n)? 　に"y"を入力
#

from gensim.models import word2vec

sentences = word2vec.LineSentence('経営学者リスト_dbpedia_wakachi.txt')
model = word2vec.Word2Vec(sentences,
                          sg=1,         #0: CBOW, 1: skip-gram
                          vector_size=300,     # ベクトルの次元数　sizeから変更
                          window=10,    # 入力単語からの最大距離
                          min_count=5,  # 単語の出現回数でフィルタリング
                          hs =1,
                          negative=0,
                          epochs=100)   # 学習を行う回数　iterから変更

model.save('./econ_word2vec')

In [8]:
model = word2vec.Word2Vec.load('./econ_word2vec')

word_vectors = model.wv.index_to_key   # どのようなタームがベクトル化されたを見る
print(word_vectors)

['、', '。', '大学', '年', '経営', '学', '（', 'の', '）', 'は', '教授', '日本', '者', '-', '・', '研究', '月', '日', '部', '経済', 'を', 'に', '名誉', '博士', '論', '院', 'と', '専門', 'で', 'し', 'た', 'て', '科', '学会', '商学', '東京', '長', '」', '「', '会長', '企業', '管理', '理事', 'など', '元', '県', '市', '出身', '賞', '会計', '一橋', '学位', '組織', '神戸', '情報', 'が', '役', '同', 'マーケティング', '法人', '学長', '助', '学院', '3', '国際', '社会', '史', '産業', '会', '大阪', '論文', 'も', '生まれ', '戦略', '取締', '2', '5', '所', '課程', '歴任', '1', '講師', '早稲田', ')', '11', '家', '会社', 'いる', '8', '中', '6', '等', '(', '委員', 'や', '12', 'ビジネス', '10', '第', '科学', '7', '4', 'マネジメント', '副', '義塾', '的', '昭和', '株式', '受賞', '慶應', '商科', '9', '政策', '学校', 'イノベーション', '県立', '卒', '静岡', '客員', '明治', '務め', '員', '経', '学者', 'システム', '京都', '学園', '卒業', 'お', 'する', '』', '『', 'か', '専攻', '協会', 'さ', '館', '工学', '法政', '文化', '中央', '定年', '士', '.', 'ある', '学術', '総合', '技術', '代表', 'センター', '准', '人', '小', '学科', '開発', '行動', '立', '工業', '北海道', '取得', 'る', '修了', 'から', 'おけ', '財団', '立命', '顧問', '分野', '専任', '関西', 'その', '地域', 'スクール', '都', 'だ',

In [9]:
# 特定の語と似通った傾向を持つ単語リストを抽出
similar_words = model.wv.most_similar(positive=["イノベーション"], topn=17)
print(*[" ".join([v, str("{:.2f}".format(s))]) for v, s in similar_words], sep="\n")

製品 0.32
ユーザー 0.29
共著 0.27
Hippel 0.26
技術 0.25
サービス 0.25
発表 0.25
von 0.24
研究 0.23
川下 0.23
Innovation 0.22
報告 0.22
附属 0.22
理科 0.22
マネジメント 0.22
貢献 0.22
決定 0.22


In [10]:
# 言葉のベクトル表現の中身を確認する
word_vector = model.wv["一橋"]
print(word_vector)

[-0.01978291  0.07069788 -0.0342469  -0.0342834   0.18341076 -0.09256226
  0.05626928  0.04721382  0.19900334  0.12484385  0.14895232 -0.17483556
  0.1722998  -0.08820935  0.1730585  -0.10900246 -0.1580718   0.03165472
  0.09999571 -0.01124644  0.00209395 -0.06861418 -0.06374303  0.00745925
 -0.05802939 -0.13828717  0.18950368 -0.1248142  -0.03546797  0.26262182
  0.18433322 -0.19910888  0.06938826 -0.21695879  0.095047   -0.07264949
 -0.08634102  0.42698348  0.07212344 -0.15643983  0.07116533 -0.10962824
 -0.12124755  0.38699344  0.09822832  0.00731103 -0.16475104 -0.18366331
 -0.00673767  0.13952273 -0.05080596  0.18181735 -0.15771365 -0.3394179
 -0.36070922  0.12143472 -0.0432054  -0.00860224  0.12853089  0.02854497
  0.25321093 -0.22734982  0.25617576 -0.26619798 -0.00852192 -0.08570755
  0.10398482 -0.02635487  0.02433784  0.02377544 -0.12264901  0.36074752
  0.23972268 -0.15306523 -0.37876707  0.11672779 -0.02549451  0.1674826
  0.11022764  0.28457862 -0.23119429  0.03853722  0.0

In [11]:
# 特定の語と語の間で似たような傾向を有するタームを取り出す
results = model.wv.most_similar(positive=[u'一橋'],negative=[u'東京'], topn=7)
for result in results:
    print(result[0])

高宮
スクール
2007
太田
再生
1988
通信


In [12]:
# 語句間の類似性について抽出
import pprint
pprint.pprint(model.wv.similarity('早稲田', '東京'))
pprint.pprint(model.wv.similarity('早稲田', '九州'))
pprint.pprint(model.wv.similarity('早稲田', 'イノベーション'))
pprint.pprint(model.wv.similarity('早稲田', '学会'))
pprint.pprint(model.wv.similarity('早稲田', '賞'))
pprint.pprint(model.wv.similarity('早稲田', '慶應'))

np.float32(0.1238754)
np.float32(0.12908997)
np.float32(-0.06876864)
np.float32(0.064702965)
np.float32(0.0022996436)
np.float32(0.1184426)
